In [1]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

In [2]:
import pandas as pd
import numpy as np 

In [3]:
df = c.data.make_base_df()
df = c.processing.transform_save_imgs(df, c.processing.grayscale_resize)

In [4]:
train_df, val_df, test_df = c.data.split_df(df)
train_df.head()

,path,type,filetype,num_id,grayscale_resize_path
137,../data/train/Type_1/441.jpg,1,jpg,441,../data/processed/grayscale_resize/train/441.jpg
130,../data/train/Type_1/396.jpg,1,jpg,396,../data/processed/grayscale_resize/train/396.jpg
1032,../data/train/Type_3/1003.jpg,3,jpg,1003,../data/processed/grayscale_resize/train/1003.jpg
177,../data/train/Type_1/623.jpg,1,jpg,623,../data/processed/grayscale_resize/train/623.jpg
1303,../data/train/Type_3/480.jpg,3,jpg,480,../data/processed/grayscale_resize/train/480.jpg


In [5]:
#test_df = c.data.make_test_df()
#test_df = c.processing.transform_save_imgs(test_df, c.processing.grayscale_resize)

In [6]:
from sklearn.ensemble import RandomForestClassifier as RFC

train_df = c.processing.random_forest_transform(train_df, 'grayscale_resize_path', grayscale=True)
test_df = c.processing.random_forest_transform(test_df, 'grayscale_resize_path', grayscale=True)


In [7]:
train_df['type'].as_matrix()

array(['1', '1', '3', '1', '3', '2', '2', '3', '3', '3', '2', '2', '3',
       '3', '1', '3', '1', '2', '3', '1', '2', '2', '2', '1', '3', '3',
       '3', '3', '2', '3', '1', '3', '2', '3', '2', '2', '2', '2', '2',
       '2', '2', '3', '1', '2', '1', '2', '3', '3', '2', '3', '2', '2',
       '3', '2', '2', '3', '3', '1', '1', '2', '2', '2', '3', '2', '3',
       '3', '3', '2', '1', '2', '2', '2', '3', '2', '1', '2', '2', '3',
       '2', '2', '1', '3', '2', '1', '3', '2', '1', '3', '2', '3', '1',
       '2', '3', '2', '2', '2', '2', '2', '3', '3', '2', '1', '3', '2',
       '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '3', '2', '2',
       '2', '2', '3', '2', '1', '2', '2', '2', '2', '3', '3', '2', '2',
       '2', '3', '2', '2', '3', '3', '3', '2', '2', '2', '1', '2', '2',
       '2', '2', '3', '3', '3', '1', '2', '3', '2', '2', '2', '1', '2',
       '3', '2', '3', '3', '1', '3', '1', '3', '3', '2', '2', '1', '2',
       '3', '3', '3', '2', '2', '2', '2', '2', '3', '3', '2', '1

In [8]:
forest = RFC(n_jobs=2,n_estimators=50)

np.squeeze(train_df['vec'].as_matrix())

#Bit of a nasty hack to get the vectors into the right form - a (888,10000) numpy array
train_vecs = np.squeeze(np.array(train_df['vec'].tolist()))
forest.fit(train)vecs, train_df['type'].as_matrix())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [9]:
test_vecs = np.squeeze(np.array(test_df['vec'].tolist()))
preds_probs = forest.predict_proba(test_vecs)

In [9]:
c.submission.write_submission_file('output2.csv', ids, preds_probs)

In [10]:
print preds_probs

[[ 0.08  0.64  0.28]
 [ 0.12  0.58  0.3 ]
 [ 0.4   0.46  0.14]
 [ 0.16  0.52  0.32]
 [ 0.16  0.48  0.36]
 [ 0.24  0.5   0.26]
 [ 0.24  0.46  0.3 ]
 [ 0.3   0.42  0.28]
 [ 0.06  0.56  0.38]
 [ 0.1   0.44  0.46]
 [ 0.12  0.38  0.5 ]
 [ 0.26  0.38  0.36]
 [ 0.18  0.52  0.3 ]
 [ 0.1   0.36  0.54]
 [ 0.08  0.54  0.38]
 [ 0.4   0.34  0.26]
 [ 0.2   0.46  0.34]
 [ 0.16  0.48  0.36]
 [ 0.16  0.58  0.26]
 [ 0.1   0.7   0.2 ]
 [ 0.32  0.42  0.26]
 [ 0.32  0.64  0.04]
 [ 0.2   0.48  0.32]
 [ 0.12  0.62  0.26]
 [ 0.24  0.46  0.3 ]
 [ 0.2   0.64  0.16]
 [ 0.2   0.48  0.32]
 [ 0.18  0.34  0.48]
 [ 0.26  0.42  0.32]
 [ 0.1   0.5   0.4 ]
 [ 0.24  0.54  0.22]
 [ 0.12  0.6   0.28]
 [ 0.38  0.46  0.16]
 [ 0.1   0.46  0.44]
 [ 0.04  0.64  0.32]
 [ 0.1   0.62  0.28]
 [ 0.02  0.5   0.48]
 [ 0.12  0.54  0.34]
 [ 0.24  0.38  0.38]
 [ 0.14  0.54  0.32]
 [ 0.12  0.4   0.48]
 [ 0.2   0.42  0.38]
 [ 0.06  0.58  0.36]
 [ 0.16  0.64  0.2 ]
 [ 0.08  0.52  0.4 ]
 [ 0.28  0.56  0.16]
 [ 0.2   0.48  0.32]
 [ 0.16  0.54